<a href="https://colab.research.google.com/github/R0N3ldrt/Thesis/blob/main/Codes/Feature_Mmimic_2.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading enviroment and required libraries 

In [1]:
# Importing necesary libraries

import math
import pickle
import os, time
import numpy as np
import collections
import pandas as pd
from os.path import exists
import matplotlib.pyplot as plt

from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor

In [2]:
# Google drive loading as work station for local-usage of the files.
from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)

#-----------------------------------------------------------------------------

Mounted at /content/gdrive


In [3]:
# Lista para cambiar los paths rapido.
workers = ["Ronald", "Local"]

# Cambiar el numero aqui para cambiar los paths.
worker = workers[0]

if worker == "Ronald":
  path= "/content/gdrive/MyDrive/Thesis_Workstation/ANN_dataset"
else: path = os.getcwd()

Pre-procesing

In [4]:
# Global variables

global distances 
distances = [80*i for i in range(1,26)]

global sc_input, sc_output
sc_input = MinMaxScaler()
sc_output = MinMaxScaler()

In [5]:
def modify_data(df, train):
  # Function that converts the provided data frame into target and explicative variables.
  colnames=df.columns[1:25]
  X=df.iloc[:,1:25]
  Y=df.iloc[:,0].to_numpy().reshape(-1,1)
  if train: 
    X = sc_input.fit_transform(X)
    Y = sc_output.fit_transform(Y)
  else: 
    X = sc_input.transform(X)
    Y = sc_output.transform(Y)
  return X,Y

In [6]:
def generate_and_train(X_train,Y_train):
  #NN Model
  model = Sequential()
  model.add(Dense(12, input_dim = 24, activation = 'tanh'))
  model.add(Dense(6,activation='tanh'))
  model.add(Dense(1))
  model.compile(loss ='mean_squared_error',optimizer = 'RMSprop')
  
  #Generate model
  model_ann = KerasRegressor(build_fn=ann,epochs=500,batch_size=32)

  #Train model
  start_time = time.time()
  callback = EarlyStopping(monitor='loss', patience=500)
  model_ann.fit(X_train, Y_train, callbacks=[callback])
  time_train_ann = time.time() - start_time
  return model_ann

In [7]:
def train_test_split_NNmodel(df_train, df_test, NN_filepath):
  X_train, Y_train = modify_data(df_train, True)
  X_test, Y_test = modify_data(df_test, False)

  
  file_exists = exists(NN_filepath)
  if file_exists != True:
    model_ann = generate_and_train(X_train,Y_train)
    pickle.dump(model_ann,open(NN_filepath,"wb"))
    print('NN models trained and saved succesfully')
  else:
    print('NN models already saved in provided filepath')
  return X_train,Y_train, X_test, Y_test

In [ ]:
def test_results(model,X_test,Y_test,log = True): 
  start_time = time.time()
  Y_test_pred=model.predict(X_test)
  time_eval_ann=time.time()-start_time

  real=list(list(zip(*Y_test))[0])

  #real=list(map(list, zip(*Y_train)))
  pred=list(Y_test_pred)
  print(pred)

  plt.plot(real,pred,'bo')
  x= np.linspace(0,1,100)
  plt.plot(x,x,"-r")
  plt.show()
  plt.close

  tot_abs, tot_square = 0, 0
  for i in range(len(real)):
    tot_abs += (real[i] - pred[i])
    tot_square += (real[i] - pred[i])**2
  if log:
    print('MAE:', tot_abs/len(real))
    print('MSE:', tot_square/len(real))
  return Y_test_pred

In [12]:
#Opening train/test and modified files
df_train=pd.read_csv(path+'/trainingData/training_data.csv')
df_test=pd.read_csv(path+'/trainingData/testing_data.csv')
df_modified = pd.read_csv(path+'/modifiedData/GD/alpha1.csv')

#Filepath of the neural network
NN_filepath = path+"/trainedModel/new_model.pkl"

#Create train test splits and train neural network if necesesary
X_train,Y_train, X_test, Y_test = train_test_split_NNmodel(df_train, df_test, NN_filepath)


NN models already saved in provided filepath
